In [ ]:

G = basis(1,0)
E = basis(2,1)

E_proj = E*E.dag()
# electronic params
epsilon = 2000.
# mode params
w1 = 100. # frequencies
w2 = 150.
gamma1 = 2*w1 # mode dissipation rates
gamma2 = 2*w2  
alpha1 = w1/10. # system-mode couplings
alpha2 = w2/10.

# build the Hamiltonian 

H_sub = epsilon*E_proj

# now we include the two modes, but don't even couple them to our electronic system
#coupling_ops = [sigma_m1.dag()*sigma_m1, sigma_m2.dag()*sigma_m2] # system-RC operators

I_sub = qeye(H_sub.shape[0])
I = enr_identity([N,N], N+exc_diff)

H_S = tensor(H_sub, I)

atemp = enr_destroy([N,N], N+exc_diff)

a_RC_exc = [tensor(I_sub, aa) for aa in atemp] # annhilation ops in exc restr basis
phonon_operators = []
for i in range(len(a_RC_exc)):
    A_i = a_RC_exc[i].dag() + a_RC_exc[i]
    H_Ii = alpha*tensor(coupling_ops[i], I)*A_i
    H_RCi = w_0*a_RC_exc[i].dag()*a_RC_exc[i]
    H_S += H_RCi
    phonon_operators.append(A_i)


def rate_up(w, T, gamma):
    n = Occupation(w, T)
    rate = 0.5 * pi * n * J(w, gamma)
    return rate

def rate_down(w, T, gamma):
    n = Occupation(w, T)
    rate = 0.5 * pi * (n + 1. ) * J(w, gamma)
    return rate

def lin_construct(O):
    Od = O.dag()
    L = 2. * spre(O) * spost(Od) - spre(Od * O) - spost(Od * O)
    return L
